In [14]:
import pandas as pd 
import numpy as np 

## 🧹 Dropping Data After Excel Cleaning
- After performing extensive data cleaning in Excel and imputing as many missing values as possible, the next step for cleaning is to drop any remaining rows that still contain missing values — specifically, cells with NaN or "No Value" — from key columns. This ensures that the dataset is reliable and ready for accurate analysis.



In [27]:
df = pd.read_excel("/Users/ayemaq/desktop/TEMPLATE_DA_Mod1Project/data/cleaned/03_Excel_Clean.xlsx", sheet_name="Cleaned_Cafe_Data")

In [16]:
df.head()

,Transaction ID,Items,Quantity,Price Per Unit,Grand total,Payment Method,Location,Transaction Date,Days_of_week,Location_was_unavailable,Unnamed: 10,Unnamed: 11,Product,Price
0,TXN_1000555,Tea,1,1.5,1.5,Credit Card,In-store,2023-10-19,1900-01-05,False,NaN,NaN,Cake,3.0
1,TXN_1001832,Salad,2,5,10,Cash,Takeaway,2023-10-19,1900-01-05,False,NaN,NaN,Coffee,2.0
2,TXN_1002457,Cookie,5,1,5,Digital Wallet,Takeaway,2023-09-29,1900-01-06,False,NaN,NaN,Cookie,1.0
3,TXN_1003246,Juice,2,3,6,NaN,NaN,2023-02-15,1900-01-04,True,NaN,NaN,Juice,3.0
4,TXN_1004184,Smoothie,1,4,4,Credit Card,In-store,2023-05-18,1900-01-05,False,NaN,NaN,Salad,5.0


In [17]:
(df.isna().sum() + (df == "No Value").sum())


Transaction ID                  0
Items                         512
Quantity                       18
Price Per Unit                 38
Grand total                    20
Payment Method               3178
Location                     3961
Transaction Date                0
Days_of_week                    0
Location_was_unavailable        0
Unnamed: 10                 10000
Unnamed: 11                 10000
Product                      9992
Price                        9992
dtype: int64

### 🗑 Dropping Rows with Missing or Invalid Entries
Dropped all rows containing NaN or "No Value" from the following critical columns to ensure data quality and consistency:
1. Items
2. Quantity
3. Price per Unit
4. Grand Total

This step helps eliminate incomplete or unreliable records before moving on to the analysis phase.



In [18]:
cols_to_clean = ['Items', 'Quantity', 'Price Per Unit', 'Grand total']


## 🧹 Attempt 1: Dropping Rows with "No Value" (Flexible Matching, Without Changing Data Types)
This code filters out rows that contain "No Value" in any of the specified important columns (like Items, Quantity, etc.) — regardless of how it's typed (e.g., "NO VALUE", " no value ", etc.). It preserves the original data types of the DataFrame.

### 🔍 Breakdown:
- df[cols_to_clean]
    - Select only the columns we're focused on cleaning (e.g., Items, Quantity, etc.).
- .apply(..., axis=1)
    - Apply the function row by row.
- lambda row: ...
    - For each row, convert each cell to a string, strip whitespace, make lowercase, and check if it equals "no value".
- .any(axis=1)
    - For each row, check: does any cell in this row equal "no value"?
- ~ (tilde)
    - Reverses the logic — instead of keeping rows with "no value", we exclude them.
- df[...]
    - Filters the DataFrame using the condition and assigns the cleaned data back to df.


In [19]:
# Step 1: Drop rows with "No Value" (case-insensitive string match)
#df = df[~df[cols_to_clean].apply( # ~ Keep only the rows that do NOT have "no value" in them.  .apply: This applies a function to each row (axis=1 = row-wise).
#    lambda row: row.astype(str).str.strip().str.lower().eq("no value"), axis=1 # This defines what to do for each row of those selected columns.
#).any(axis=1)]


In [20]:
# Step 2: Drop NaNs in those columns
#df.dropna(subset=cols_to_clean, inplace=True)

In [21]:
# Step 3: Drop rows with 0 in Quantity, Price Per Unit, or Grand total
#df = df[~((df['Quantity'] == 0) | (df['Price Per Unit'] == 0) | (df['Grand total'] == 0))]

In [22]:
# Step 4: Reset index
# df.reset_index(drop=True, inplace=True)

## Attempt 2: Dropping Rows Based on the Number of "Bad" Values (
This approach counts how many "bad values" (like NaN, "no value", or blank entries) exist per row in the four most important columns:
- Items, Quantity, Price Per Unit, and Grand total.
- Then we remove rows that have 2 or more bad values (i.e., we keep rows that have at least 3 good values).e can also confirm that the code is working as expected by printing how many rows were dropped.

In [23]:
'''cols_to_check = ["Items", "Quantity", "Price Per Unit", "Grand total"]

# Check how many 'bad' values are in each row for those 4 columns
bad_counts = df[cols_to_check].apply(
    lambda row: sum((pd.isna(row)) | (row.astype(str).str.strip().str.lower() == "no value")),
    axis=1
)

# Count how many rows had 3 or more bad values
print("Rows with 2 or more bad values in important columns:", (bad_counts >= 2).sum())

# df = df[bad_counts < 2]  # keep rows with 0, 1, or bad values
'''

'cols_to_check = ["Items", "Quantity", "Price Per Unit", "Grand total"]\n\n# Check how many \'bad\' values are in each row for those 4 columns\nbad_counts = df[cols_to_check].apply(\n    lambda row: sum((pd.isna(row)) | (row.astype(str).str.strip().str.lower() == "no value")),\n    axis=1\n)\n\n# Count how many rows had 3 or more bad values\nprint("Rows with 2 or more bad values in important columns:", (bad_counts >= 2).sum())\n\n# df = df[bad_counts < 2]  # keep rows with 0, 1, or bad values\n'

### Attempt 3: Drop Rows Only When They Are Likely Unusable
In this logic, we drop a row only if both of the following conditions are met:
- Either Payment Method or Location is blank, AND There are 2 or more “bad” values in the important columns:
    - Items, Quantity, Price Per Unit (PPU), and Grand total.
- “Bad” values include:
    - "No value"
    - 0
    - Empty cells
    - NaN


EXAMPLE


| Item | Quantity | PPU | GT | Location | PM       | Action                                       |
| ---- | -------- | --- | -- | -------- | -------- | -------------------------------------------- |
| 0    | NA       | 2   | 2  | No value | No value | ✅ Drop (2+ bad values + missing location/PM) |
| 1    | 0        | 2   | 2  | No value | No value | ❌ Keep (only 1 bad value in important cols)  |



In [24]:
cols_to_check = ["Items", "Quantity", "Price Per Unit", "Grand total"]

def is_blank(val):
    return pd.isna(val) or str(val).strip() == ""

df = df[~df.apply(
    lambda row: (
        # Check if either Payment Method or Location is blank
        is_blank(row["Payment Method"]) or is_blank(row["Location"])
    ) and (
        # Check if 2 or more of the important columns are bad
        sum(is_blank(row[col]) or str(row[col]).strip().lower() == "no value" or row[col] == 0 for col in cols_to_check) >= 2
    ),
    axis=1
)]


In [25]:
# to check
# Redefine is_blank for use here too
def is_blank(val):
    return pd.isna(val) or str(val).strip() == ""

# Check for rows that should have been dropped — sanity check
bad_rows = df[df.apply(
    lambda row: (
        is_blank(row["Payment Method"]) or is_blank(row["Location"])
    ) and (
        sum(is_blank(row[col]) or str(row[col]).strip().lower() == "no value" or row[col] == 0 for col in cols_to_check) >= 2
    ),
    axis=1
)]

# Print how many got through (should be 0)
print("Rows that still match the bad condition:", len(bad_rows))


Rows that still match the bad condition: 0


In [26]:
df.to_excel("/Users/ayemaq/desktop/TEMPLATE_DA_Mod1Project/data/cleaned/cafe_data.xlsx", index=False)
